In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import time
import pyautogui

import os
import threading
import dpkt

import collections as clt

In [31]:
err_list = list()

input_path = 'F:/data_edge/2_pcap_ow/'

count_url = 1100
count_repeat = 1

for A in range(100, count_url):
    
    for B in range(count_repeat):
        
        file_name = input_path + str(A) + '-' + str(B) + '.pcap'
        
        print('\rprocessing:' + str(A) + '-' + str(B), end='')
        
        try:
            with open(file_name, 'rb') as f:

                pcap = dpkt.pcap.Reader(f)

                data = list()

                for ptime, pdata in pcap:

                    data.append(len(pdata))
                
                if len(data) < 10:
                    print('\tlenerr')
                    err_list.append([A, B])
                
        except Exception as e:
            print('\topenerr')
            err_list.append([A, B])
            pass
                
print('\ndone!')

processing:1089-0
done!


In [32]:
len(err_list)

0

In [33]:
err_list

[]

In [28]:
def windump_thread(file_name, filt):
    os.system('windump -i 1 -s 65535 -w ' + file_name + ' src or dst ' + filt)
    
def visit_thread(url):
    os.system('msedge ' + url)

In [29]:
url_path = 'F:/data_edge/1_url/url.csv'
out_path = 'F:/data_edge/2_pcap_ow/'

df_url = pd.read_csv(url_path, header=None)

count = 0

for A, B in err_list:
    
    url = df_url.iloc[A, 1]

    count += 1
    print('\r                                                          ', end='')
    print('\rcapturing:', A, '-', B, end='  ')
    print(str(count) + '/' + str(len(err_list)), end='  ')
    print(url, end='  ')

    t1 = threading.Thread(target=windump_thread, 
                          args=(out_path + str(A) + '-' + str(B) + '.pcap', url))
    t1.start()

    time.sleep(1)

    t2 = threading.Thread(target=visit_thread, 
                          args=(url,))
    t2.start()

    time.sleep(3)
    loading_time = 0
    while(pyautogui.pixel(755,60) == (30,30,30)):
        time.sleep(1)
        loading_time += 1
        if loading_time >= 20:
            break

    time.sleep(1)

    pyautogui.screenshot(out_path + str(A) + '-' + str(B) + '.jpg')

    os.system('taskkill /F /IM windump.exe')
    os.system('taskkill /F /IM msedge.exe')

    time.sleep(1)
    
print('\ndone!')

capturing: 1089 - 0  6/6  news.jxcn.cn                    
done!


In [ ]:
import os
os.system('taskkill /F /IM windump.exe')